In [1]:
!pip install deep-hedging

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
TILL_MATURITY = 3.0
N_DAYS = 252 * TILL_MATURITY

LOWER_STRIKE = 100 / 100
UPPER_STRIKE = 112 / 100
PARTICIPATION = 2

RF_RATE = 0.03
VOL = 0.2

In [4]:
DESIRED_PAYOFF = lambda spot: PARTICIPATION * np.maximum(
    0, np.minimum(spot[:, -1] - 1, UPPER_STRIKE - LOWER_STRIKE)
)

In [5]:
start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

underlyings = Underlyings(
    tickers=[Ticker(f"Some Stock", f"STNK")],
    start=start,
    end=end,
    dividends=np.array([0.0]),
    means=np.array([RF_RATE]),
    var_covar=np.array([VOL**2]),
)
underlyings.data

In [6]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

In [7]:
call_lower = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=LOWER_STRIKE,
    start_date=start,
    end_date=end,
)
call_upper = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=UPPER_STRIKE,
    start_date=start,
    end_date=end,
)

call_spread = (call_lower - call_upper) * PARTICIPATION
call_spread

StructuredNote of:
1. LONG 2.0 units of EuropeanCall:
* Underlyings:
-> Ticker(name='Some Stock', code='STNK', currency=None)
* Strike = 100.0
* Start Date = 2019-01-01 00:00:00
* End Date = 2021-01-26 00:00:00
.

2. SHORT 2.0 units of EuropeanCall:
* Underlyings:
-> Ticker(name='Some Stock', code='STNK', currency=None)
* Strike = 112.00000000000001
* Start Date = 2019-01-01 00:00:00
* End Date = 2021-01-26 00:00:00
.


In [8]:
pricer = GBMPricer(payoff_function=call_spread.payoff, random_seed=RANDOM_SEED)
paths = pricer.get_paths(
    spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2] * len(t)),
).squeeze(2)
paths.shape

(100000, 757)

In [9]:
assert np.allclose(call_spread.payoff(paths)[:, -1], DESIRED_PAYOFF(paths), atol=1e-6)